# Librerías

In [1]:
import itertools
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from sklearn import (manifold, datasets, decomposition, ensemble,
                     discriminant_analysis, random_projection)#Prueba evaluación y entrenamiento un solo individuo. 
#Vectores completos
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import random

In [2]:
import seaborn as sns

# Databases

In [3]:
#Reading all the cvs
#Nss learning
nsl_training=pd.read_csv('./Final_Datasets/MINMAX_FD_NSL_Tra.csv')
#Nsl testing
nsl_testing=pd.read_csv('./Final_Datasets/MINMAX_FD_NSL_Test.csv')

# Labels

In [4]:
#NSL TESTING LABELS

#Index binary (2 clases)
L_nsl_testing_indexbinary=pd.read_csv("./Final_Datasets/L_nsl_testing_indexbinary.csv")
#Index by Group (5 clases)
L_nsl_testing_indexbygroup=pd.read_csv("./Final_Datasets/L_nsl_testing_indexbygroup.csv")
#Index tipe of attack (42? clases)
L_nsl_testing_indextipeofattack=pd.read_csv("./Final_Datasets/L_nsl_testing_indextipeofattack.csv")


#nsl TRAINING LABELS
# 2 clases
L_nsl_training_indexbinary=pd.read_csv("./Final_Datasets/L_nsl_training_indexbinary.csv")
#Index by Group (5 clases)
L_nsl_training_indexbygroup=pd.read_csv("./Final_Datasets/L_nsl_training_indexbygroup.csv")
#Index tipe of attack (42? clases)
L_nsl_training_indextipeofattack=pd.read_csv("./Final_Datasets/L_nsl_training_indextipeofattack.csv")

# Conversión de Pandas->Numpy (array) not Matrix

In [5]:
nsl_training=nsl_training.as_matrix(columns=None)
nsl_testing=nsl_testing.as_matrix(columns=None)

# Funcion para calcular M dependiendo de las iteraciones

In [6]:
def M_dimensiones_a_mutar(Perror,i_actual,i_max,M_Max=40):
    iteraciones=1-(i_actual/i_max)
    M=M_Max*np.amin([iteraciones,Perror])
    M=np.array(np.round_(M))
    return M

# Funcion para calcular el rendimiento F(s)

In [7]:
def funcion_rendimiento(Perror_actual,unos,N=40,alfa=.3):
    cardinalidad=unos
    rendimiento=alfa*Perror_actual+(1-alfa)*(cardinalidad/N)
    return rendimiento

# Declaración del modelo

In [8]:
from sklearn.ensemble import RandomForestClassifier

modelo=RandomForestClassifier(criterion='entropy',n_jobs=2, 
                              bootstrap=True, max_depth=10, max_features='sqrt', min_samples_leaf=2, 
                              min_samples_split=100, n_estimators=8)


# Declarando e modelo para la distancia

In [9]:
from sklearn.neighbors import DistanceMetric
dist = DistanceMetric.get_metric('jaccard')

# IMO

# Funciones

# Eval sencilla

In [10]:
#funcion para la evaluacion sencilla de un elemento
#la evaluación sencilla consta de 
#indices_uno_poblacion, guarda los indices de la poblacion de aniones o cationes las cuales contienen los
#parametros de 1
def evaluacion_sencilla(poblacion,matriz_de_evaluacion,indices_unos_poblacion,modelo):
    """Regresa la matriz dd resultados
    los indices dónde se encuentran los unos"""
    for indice_elemento in list(range(0,(poblacion.shape[0]))):
        #filtrado de la base de datos
        training_temp=poblacion[indice_elemento]*nsl_training
        testing_temp=poblacion[indice_elemento]*nsl_testing    
        #se entrena el modelo
        modelo.fit(training_temp,training_etiquetas.values.ravel())
        #se evalua el modelo
        predicciones_temp=modelo.predict(testing_temp)
        #se obtiene el accuracy score
        matriz_de_evaluacion[indice_elemento,0]=accuracy_score(testing_etiquetas,predicciones_temp)
        #se obtiene la probabilidad de error
        matriz_de_evaluacion[indice_elemento,1]=1-accuracy_score(testing_etiquetas,predicciones_temp)
        #se obtienen la cantidd de los indices de los unos (dimensinoes activas)
        matriz_de_evaluacion[indice_elemento,2]=np.array(np.where(poblacion[indice_elemento]==1)).shape[1]
        #se obtienen los indices de los unos
        indices_unos_poblacion.append(np.where(poblacion[indice_elemento]==1))
        
        print("Elemento: %s, unos temporales: %s" %(indice_elemento,M_E_A[indice_elemento,2]))
    return [matriz_de_evaluacion,indices_unos_poblacion]

        

# Eval compuesta

In [11]:

#Matriz con resultados para su actualizacion
#0-column (distancia - Jaccard)
#1-column (fuerza)
#2-column (cantidad de dimensiones similares con el mejor)

#funcion para la evaluacion compuesta de un elemento
#la evaluación sencilla consta de 
def evaluacion_compuesta(poblacion,mejor_elemento_pob_contraria,matriz_de_evaluacion):
    """Regresa la matriz dd resultados compuestos, la cual incluye
    distancia (jacard), fuerza, y los indices de los vectores iguales entre 
    el elemento x y el mejor de la población"""
    for indice_elemento in list(range(0,(poblacion.shape[0]))):
        #se calcula el indice [elemento,3]->Distancia
        a=poblacion[indice_elemento]
        b=mejor_elemento_pob_contraria
        matriz_de_evaluacion[indice_elemento,3]=dist.pairwise([a,b])[0,1]
        #se calcula el indice [elemento,4]->fuerza
        x=matriz_de_evaluacion[indice_elemento,3]
        fuerza=(1/(1+(2.71828**(-0.1/x))))
        #se almacena la fuerza en la matriz
        matriz_de_evaluacion[indice_elemento,4]=fuerza
        #se calcula el rendimiento F(S)
        #se calcula el indice [elemento,5]->Rendimiento
        matriz_de_evaluacion[indice_elemento,5]=funcion_rendimiento(matriz_de_evaluacion[indice_elemento,1],
                                matriz_de_evaluacion[indice_elemento,2],N=40,alfa=.3)
                             
 
        print("indice_elemento %s" %indice_elemento)
    return [matriz_de_evaluacion]


# Fase líquida

In [12]:
def fase_liquida(poblacion,mejor_pob_contraria,matriz_resultados,i_actual,i_max):
    """poblacion=poblacion de aniones/cationes
    mejor_pob_contraia=mejor elemento de la poblacion contraria a partir de la cual mutar
    fase=liquida,solida"""
    #probando la fase líquida
    for indice_elemento in list(range(0,(poblacion.shape[0]))):
        
        print("\n")
        print("Me encuentro trabajando en el elmento %s de %s totales" %(indice_elemento,poblacion.shape[0]))
        respaldo_temporal_elemento=np.array(poblacion[indice_elemento])
        print("Elemento:")
        print(respaldo_temporal_elemento)
        
        
        #Encuentra el total de dimensiones M posibles a mutar POR ELEMENTO
        M=M_dimensiones_a_mutar(matriz_resultados[indice_elemento,1],i_actual,i_max)
        
        print("Tiene una probabilidad de error de: %s" % matriz_resultados[indice_elemento,1])
        print("Para el elemento %s se mutan un total máximo de M=%s dimensiones" %(indice_elemento,M))
        
        #Encuentra el total de dimensinoes posibles a mutar, dependiendo del elemento (distancia,fuerza)
        #y que ademas depende de M
        #posicion_temp=cantidad de dimensinoes a mutar por elemento. 
        
        #matriz_resultados[elemento,5]->posicion
        #matriz_resultados[elemnto,4]->fuerza

        
        #posicion temporal, maxima cantidad de dimensinoes permitidas a mutar por ese elemento
        posicion_temp=matriz_resultados[indice_elemento,4]*M
        #se redondea para que se un total de dimensiones entero
        posicion_temp=int(np.round_(posicion_temp))
        
        print("Cantidad de dimensiones a mutar específicas para ese elemento: %s" %(posicion_temp))
     
        
        #se actualiza la matriz de resultados
        matriz_resultados[indice_elemento,6]=posicion_temp
        
        #muta unicamente los iguales, aún pese a que haya más elementos posibles a mutar M
        #debido a la posicion_temp
    
        a=np.where((poblacion[indice_elemento]==mejor_pob_contraria)==False)
        #para trabajar sin dificultades se convierte a una lista plana
        b=np.concatenate(a).ravel().tolist()
                
        print("El elemento mejor es: ")
        print(mejor_pob_contraria)
        print("El elemento a evaluar es: ")
        print(poblacion[indice_elemento])
        print("Los indices que son diferentes son %s" %(a))
        
        #se seleccionan posicion_temp elementos de forma ALEATORIA, para no elegirse siempre las primeras
        #dimensiones
        
        dimensiones_utilizadas=0
        criterio_fase_solida=0
        
        
        #--------------------------------
        
        if (len(b)!=0):
            if (posicion_temp>len(b)):
                #solución al problema de cuando hay más cantidad de 
                #dimensiones posibles que de elementos, es decir,
                #se toma todo el conjunto de dimensiones iguales
                print("Entre al primer if")
                diferentes_temp_indices=b
                diferentes_temp_cantidad=len(diferentes_temp_indices)
                
                
                print("indice_elemento: %s" %indice_elemento,"diferentes temp indices", diferentes_temp_indices,
                     "dimensinoes utilizadas: %s" %dimensiones_utilizadas)

            else:
                
                print("Entre al primer else")
                #cuando hay más dimensiones diferentes que dimensiones posibles a mutar
                #se escogen de manera aleatoria
                c=random.sample(b,posicion_temp)
                diferentes_temp_indices=sorted(c)
                diferentes_temp_cantidad=len(diferentes_temp_indices)

                
                print("indice_elemento: %s" %indice_elemento,"diferentes temp indices", diferentes_temp_indices,
                     "dimensinoes utilizadas: %s" %dimensiones_utilizadas)

                
            #while para mutar las dimensiones diferentes
            #se resta uno por el while, si son 5, toma 6, debido a que empieza en cero
            while((dimensiones_utilizadas<diferentes_temp_cantidad)):
                print("Entre al primer while")
                print("Diferentes temp cantidad %s" %diferentes_temp_cantidad)
                #muta los diferentea
                #print("\n\n")
                #print(diferentes_temp_indices)
                #print(dimensiones_utilizadas)
                #print(type(diferentes_temp_indices))
                #print(diferentes_temp_indices[0,dimensiones_utilizadas])
                
                print("indice_elemento: %s" %indice_elemento,"diferentes temp indices", diferentes_temp_indices,
                     "dimensinoes utilizadas: %s" %dimensiones_utilizadas)

                if (poblacion[indice_elemento,diferentes_temp_indices[dimensiones_utilizadas]] ==0):
                    poblacion[indice_elemento,diferentes_temp_indices[dimensiones_utilizadas]] =1
                else:
                    poblacion[indice_elemento,diferentes_temp_indices[dimensiones_utilizadas]] =0


                print("Se muto la dimension: %s" %(diferentes_temp_indices[dimensiones_utilizadas]))
                print("El elemento de la poblacion %s mutado es: ")
                print(poblacion[indice_elemento])

                dimensiones_utilizadas=dimensiones_utilizadas+1      
                
                
        else:
            #poblacion[elemento,7]->1, si todas las dimensiones son diferentes; 
            #0 en caso contrario
            poblacion[indice_elemento,7]=1
            #cuando son exactamente iguales el criterio para la fase sólida aumenta
            #
    
    return [poblacion,matriz_resultados]
            





# Mejor individuo

In [13]:
def mejor_individuo(poblacion_matriz_evaluacion):
    mejor=np.argmin(poblacion_matriz_evaluacion[:,5])
    return mejor
    

# Fase SOLIDA

In [14]:
def fase_solida(poblacion,mejor_pob_contraria,matriz_resultados,i_actual,i_max):
    """poblacion=poblacion de aniones/cationes
    mejor_pob_contraia=mejor elemento de la poblacion contraria a partir de la cual mutar
    fase=liquida,solida"""
    #probando la fase líquida
    for indice_elemento in list(range(0,(poblacion.shape[0]))):
        
        print("\n")
        print("Me encuentro trabajando en el elmento %s de %s totales" %(indice_elemento,poblacion.shape[0]))
        respaldo_temporal_elemento=np.array(poblacion[indice_elemento])
        print("Elemento:")
        print(respaldo_temporal_elemento)
        
        
        #Encuentra el total de dimensiones M posibles a mutar POR ELEMENTO
        M=M_dimensiones_a_mutar(matriz_resultados[indice_elemento,1],i_actual,i_max)
        
        print("Tiene una probabilidad de error de: %s" % matriz_resultados[indice_elemento,1])
        print("Para el elemento %s se mutan un total máximo de M=%s dimensiones" %(indice_elemento,M))
        
        #Encuentra el total de dimensinoes posibles a mutar, dependiendo del elemento (distancia,fuerza)
        #y que ademas depende de M
        #posicion_temp=cantidad de dimensinoes a mutar por elemento. 
        
        #matriz_resultados[elemento,5]->posicion
        #matriz_resultados[elemnto,4]->fuerza

        
        #-Fase Sólida-#
        
        r=random.random()
        
        if r>0.5:
            print("Separción Drástica")
            
            #separacion drástica
            #posicion temporal, maxima cantidad de dimensinoes permitidas a mutar por ese elemento
            posicion_temp=matriz_resultados[indice_elemento,4]*M
            #se redondea para que se un total de dimensiones entero
            posicion_temp=int(np.round_(posicion_temp))
            
        else:
            print("separacion mesurada")
            #Separacion Mesurada
            alfa=random.uniform(.5,1)
            #posicion temporal, maxima cantidad de dimensinoes permitidas a mutar por ese elemento
            posicion_temp=matriz_resultados[indice_elemento,4]*M*alfa
            #se redondea para que se un total de dimensiones entero
            posicion_temp=int(np.round_(posicion_temp))

        #-Fin fase Solida-#
        
        print("Cantidad de dimensiones a mutar específicas para ese elemento: %s" %(posicion_temp))
     
        
        #se actualiza la matriz de resultados
        matriz_resultados[indice_elemento,6]=posicion_temp
        
        #muta unicamente los iguales, aún pese a que haya más elementos posibles a mutar M
        #debido a la posicion_temp
    
        a=np.where((poblacion[indice_elemento]==mejor_pob_contraria)==True)
        #para trabajar sin dificultades se convierte a una lista plana
        b=np.concatenate(a).ravel().tolist()
                
        print("El elemento mejor es: ")
        print(mejor_pob_contraria)
        print("El elemento a evaluar es: ")
        print(poblacion[indice_elemento])
        print("Los indices que son iguales son %s" %(a))
        
        #se seleccionan posicion_temp elementos de forma ALEATORIA, para no elegirse siempre las primeras
        #dimensiones
        
        dimensiones_utilizadas=0
        criterio_fase_solida=0
        
        
        #--------------------------------
        
        if (len(b)!=0):
            if (posicion_temp>len(b)):
                #solución al problema de cuando hay más cantidad de 
                #dimensiones posibles que de elementos, es decir,
                #se toma todo el conjunto de dimensiones iguales
                print("Entre al primer if")
                diferentes_temp_indices=b
                diferentes_temp_cantidad=len(diferentes_temp_indices)
                
                
                print("indice_elemento: %s" %indice_elemento,"iguales temp indices", diferentes_temp_indices,
                     "dimensinoes utilizadas: %s" %dimensiones_utilizadas)
                

            else:
                
                print("Entre al primer else")
                #cuando hay más dimensiones diferentes que dimensiones posibles a mutar
                #se escogen de manera aleatoria
                c=random.sample(b,posicion_temp)
                diferentes_temp_indices=sorted(c)
                diferentes_temp_cantidad=len(diferentes_temp_indices)
                

                
                print("indice_elemento: %s" %indice_elemento,"iguales temp indices", diferentes_temp_indices,
                     "dimensinoes utilizadas: %s" %dimensiones_utilizadas)

                
            #while para mutar las dimensiones diferentes
            #se resta uno por el while, si son 5, toma 6, debido a que empieza en cero
            while((dimensiones_utilizadas<diferentes_temp_cantidad)):
                print("Entre al primer while")
                print("Diferentes temp cantidad %s" %diferentes_temp_cantidad)
                #muta los diferentea
                #print("\n\n")
                #print(diferentes_temp_indices)
                #print(dimensiones_utilizadas)
                #print(type(diferentes_temp_indices))
                #print(diferentes_temp_indices[0,dimensiones_utilizadas])
                
                print("indice_elemento: %s" %indice_elemento,"iguales temp indices", diferentes_temp_indices,
                     "dimensinoes utilizadas: %s" %dimensiones_utilizadas)

                if (poblacion[indice_elemento,diferentes_temp_indices[dimensiones_utilizadas]] ==0):
                    poblacion[indice_elemento,diferentes_temp_indices[dimensiones_utilizadas]] =1
                else:
                    poblacion[indice_elemento,diferentes_temp_indices[dimensiones_utilizadas]] =0


                print("Se muto la dimension: %s" %(diferentes_temp_indices[dimensiones_utilizadas]))
                print("El elemento de la poblacion %s mutado es: ")
                print(poblacion[indice_elemento])

                dimensiones_utilizadas=dimensiones_utilizadas+1      
                
                
        
    return [poblacion,matriz_resultados]
            





# Dos posibilidades <n>
## 1) guardar todos los mejores resultados de cada poblacion por iteracion. Un resultado por iteracion entre cationes y aniones
## 2) guardar unicamente los espacios de búsqueda que cumplan con F(temp)<Fbest (mejor solución actul que la anterior)

## probando la primer posibilidad

In [15]:
#etiquetas para el modelo
training_etiquetas=L_nsl_training_indexbinary
testing_etiquetas=L_nsl_testing_indexbinary

# Funcion general IMO

In [21]:
#Registros de resultados GLOBALES
resultados_score=[]
resultados_cantidad_dimensiones=[]
resultados_rendimiento=[]
resultados_dimensiones_activas=[]
resultados_indices_dimensiones_activas=[]

#while
iteracion_general=0
maxima_cantidad_de_corridas=1
start_time = time.time()

while iteracion_general<maxima_cantidad_de_corridas:
    print("\n\n Corrida %s " % maxima_cantidad_de_corridas)
    
    i=0
    M=38
    iteraciones_maximas=4
    

    #REGISTRO DE RESULTADOS POR EJECUCION DE EJERCICIO (300 iteraciones)
    
    #registro de la probabilidad de error de los valores aceptados
    registro_perror_valores_aceptados=[]
    #almacena unicamente el score de los vectores binarios aceptados
    registro_score_valores_aceptados=[]
    #almacena el vector binario de los valores aceptados (dimensiones_activas)
    #es un sinonimo de la mejor solucion del espacio de búsqueda
    registro_dimensiones_activas_valores_aceptados=[]
    #registro cantidad de dimensiones activas de los valores aceptados
    registro_cantidad_dimensiones_activas_aceptados=[]
    #registro del rendimiento de los valore aceptados
    registro_rendimiento_aceptados=[]    
    #registro de los unos
    registro_indices_dimensiones_activas=[]
    
    #1) Declaración de parámetros específicos para IMO
    
    
    #INDICES
    m_c=0 #mejor cation
    m_a=0 #menor anion
    m_m=0 #mejor elemento entre aniones y cationes
    m_e=0 #mejor elemento hasta el momento entre todas las iteraciones (best)
    
    
    #Declaracion de la poblacion de forma aleatoria

    A=np.random.randint(2, size=(10,40))
    C=np.random.randint(2, size=(10,40))

    #Matriz Dimensiones activas (lista) (dimensiones unos):
    #dimensiones_activas_aniones=[]
    #dimensiones_activas_cationes=[]

    I_A=[]
    I_C=[]
    I_M=[]
    
    #Matriz de evaluacion con todos los parámetros
    M_E_A=np.zeros([10,8])
    M_E_C=np.zeros([10,8])


    
    #####---#####---####
    #almacena UNICAMENTE el mejor entre ANION y CATION
    #Cuando se cumple la condicion que el nuevo ANION/CATION
    #da un rendimiento MENOR al mejor anterior
    
    
    #Evaluar el primer elemento, y guardarlo, asumiento que el primero es el mejor
    #PRIMER ELEMENTO
    
    
    #evaluar todo->guardarlo->aplicar fase liquida/solida (para que despues sea vuelto a evaluar)
    
    #Evaluación Sencilla
    #Población Aniones
    [M_E_A,I_A]=evaluacion_sencilla(A,M_E_A,I_A,modelo)
    #Poblacion Cationes
    [M_E_C,I_C]=evaluacion_sencilla(C,M_E_C,I_C,modelo)


    #encontrando los individuos
    m_a=mejor_individuo(M_E_A)
    m_c=mejor_individuo(M_E_C)        

    #Evaluación compuesta
    #Poblacion Aniones
    [M_E_A]=evaluacion_compuesta(A,C[m_c],M_E_A)
    #Poblacion Cationes
    [M_E_C]=evaluacion_compuesta(C,A[m_a],M_E_C)
    
    #Encontrando al mejor de las dos poblaciones
    
    if (M_E_A[m_a,5] < M_E_C[m_c,5]):        
        #el anion es mejor solucion
        m_m=m_a
        #M_M_T_R=mejor poblacion temporal (matriz de resultados)
        #M_M_T=mejor poblacion temporal 
        M_M_T=np.array(A)
        M_M_T_R=np.array(M_E_A)
    else:
        #el cation es la mejor solucion
        m_m=m_c
        M_M_T=np.array(C)
        M_M_T_R=np.array(M_E_C)

    #almacenando los valores
    
    
    #Se usa M_M_T_R porque son los RESULTADOS
    #5->rendimiento
    registro_rendimiento_aceptados.append(M_M_T_R[m_m,5])
    #0->score
    registro_score_valores_aceptados.append(M_M_T_R[m_m,0])
    #1->perror
    registro_perror_valores_aceptados.append(M_M_T_R[m_m,1])
    #Nuevo mejor elemento de ambas poblaciones (total de dimensiones)
    #M_M_T porque es la mejor poblacion, anion o cation
    registro_dimensiones_activas_valores_aceptados.append(M_M_T[m_m])
    #Registro de la cantidad de unos
    #se obtienen los indices del los unos (dimensiones activas)
    registro_indices_dimensiones_activas.append(np.where(M_M_T[m_m]==1))        
    #se obtiene el total de dimensiones activas
    registro_cantidad_dimensiones_activas_aceptados.append(np.array(np.where(M_M_T[m_m]==1)).shape[1])


    #aplicando la fase líquida en la primer evaluación
    
    print("Entre en fase de la primera evaluación")
    [A,M_E_A]=fase_liquida(A,C[m_c],M_E_A,i,iteraciones_maximas)
    #cationes
    [C,M_E_C]=fase_liquida(C,A[m_a],M_E_C,i,iteraciones_maximas)
    
    
    #evalua de forma general


    while (i<iteraciones_maximas):
        print("\n\n Iteracion: %s de %s \n\n" %((i),iteraciones_maximas))     
        #Encontrando los criterios de evaluación
        criterio_A=np.where(M_E_A[:,4]>=.6)
        criterio_A=np.concatenate(criterio_A).ravel().tolist()
        criterio_C=np.where(M_E_A[:,4]>=.6)
        criterio_C=np.concatenate(criterio_C).ravel().tolist()
        
        
        #evaluando las poblaciones
        
        #Evaluación Sencilla
        #Población Aniones
        [M_E_A,I_A]=evaluacion_sencilla(A,M_E_A,I_A,modelo)
        #Poblacion Cationes
        [M_E_C,I_C]=evaluacion_sencilla(C,M_E_C,I_C,modelo)

        
        #encontrando los individuos
        m_a=mejor_individuo(M_E_A)
        m_c=mejor_individuo(M_E_C)        
        
        #Evaluación compuesta
        #Poblacion Aniones
        [M_E_A]=evaluacion_compuesta(A,C[m_c],M_E_A)
        #Poblacion Cationes
        [M_E_C]=evaluacion_compuesta(C,A[m_a],M_E_C)
        
        
        #Aquí se almacenan los resultados. Antes
        #De que resulten ser mutados nuevamente. por
        #las fases líquida/solida
        
        #Encuentra al mejor entre las dos poblaciones
        
        
        if (M_E_A[m_a,5] < M_E_C[m_c,5]):
            #el anion es mejor solucion
            m_m=m_a
            #M_M=mejor poblacion temporal
            M_M_T=np.array(A)
            M_M_T_R=np.array(M_E_A)
            print("El mejor elemento es un anion: ", M_M_T[m_m])  
              
        else:
            #el cation es la mejor solucion
            m_m=m_c
            M_M_T=np.array(C)
            M_M_T_R=np.array(M_E_C)
            print("El mejor elemento es un cation: ", M_M_T[m_m])  
            
        
        #if el nuevo mejor tiene un rendimiento mejor que el mejor anterior
        #actualiza y salva todos los nuevos valores
        if M_M_T_R[m_m,5]<registro_rendimiento_aceptados[-1]:
              
            print("SI SE encontró una mejor solución:")
            print("Solución anterior: ", registro_rendimiento_aceptados[-1])
            print("Solución actual: ", M_M_T_R[m_m,5])
            
            #Se usa M_M_T_R porque son los RESULTADOS
            #5->rendimiento
            registro_rendimiento_aceptados.append(M_M_T_R[m_m,5])
            #0->score
            registro_score_valores_aceptados.append(M_M_T_R[m_m,0])
            #1->perror
            registro_perror_valores_aceptados.append(M_M_T_R[m_m,1])
            #Nuevo mejor elemento de ambas poblaciones (total de dimensiones)
            #M_M_T porque es la mejor poblacion, anion o cation
            registro_dimensiones_activas_valores_aceptados.append(M_M_T[m_m])
            #Registro de la cantidad de unos
            #se obtienen los indices del los unos (dimensiones activas)
            registro_indices_dimensiones_activas.append(np.where(M_M_T[m_m]==1))        
            #se obtiene el total de dimensiones activas
            registro_cantidad_dimensiones_activas_aceptados.append(np.array(np.where(M_M_T[m_m]==1)).shape[1])
            
        else:
            print("NO SE encontró mejor solución")
            print("Solución anterior: ", registro_rendimiento_aceptados[-1])
            print("Solución actual: ", M_M_T_R[m_m,5])
        #sino es mejor, discrimina esa ultima solucion
        
        
        
        #IF para entrar en la fase líquida o sólida
        
        if ((int(len(criterio_A)) <5) or (int(len(criterio_C)) <5)):
            #entra en FASE LÍQUIDA
            #evaluación
            #aniones
            print("Entre en fase LIQUIDA")
            [A,M_E_A]=fase_liquida(A,C[m_c],M_E_A,i,iteraciones_maximas)
            #cationes
            [C,M_E_C]=fase_liquida(C,A[m_a],M_E_C,i,iteraciones_maximas)
        else:
            #entra en Fase Sólida
            #aniones
            print("Entre en fase SOLIDA")
            [A,M_E_A]=fase_solida(A,C[m_c],M_E_A,i,iteraciones_maximas)
            [C,M_E_C]=fase_solida(C,A[m_a],M_E_C,i,iteraciones_maximas)
        
        
    #iteraciones, por ejercicio

        i=i+1
    
    
    
    #FIN corrida, se almacenan los resultados de dicha corrida.
    
    resultados_score.append(registro_score_valores_aceptados)
    resultados_cantidad_dimensiones.append(registro_cantidad_dimensiones_activas_aceptados)
    resultados_rendimiento.append(registro_rendimiento_aceptados)
    resultados_dimensiones_activas.append(registro_dimensiones_activas_valores_aceptados)
    resultados_indices_dimensiones_activas.append(registro_indices_dimensiones_activas)
    
    iteracion_general=iteracion_general+1
t_entrenamiento_f_o=(time.time() - start_time)


print("\n Tiempo de IMO\n")
print("--- %s seconds ---" % t_entrenamiento_f_o)



 Corrida 1 
Elemento: 0, unos temporales: 17.0
Elemento: 1, unos temporales: 22.0
Elemento: 2, unos temporales: 21.0
Elemento: 3, unos temporales: 23.0
Elemento: 4, unos temporales: 19.0
Elemento: 5, unos temporales: 17.0
Elemento: 6, unos temporales: 19.0
Elemento: 7, unos temporales: 20.0
Elemento: 8, unos temporales: 21.0
Elemento: 9, unos temporales: 23.0
Elemento: 0, unos temporales: 17.0
Elemento: 1, unos temporales: 22.0
Elemento: 2, unos temporales: 21.0
Elemento: 3, unos temporales: 23.0
Elemento: 4, unos temporales: 19.0
Elemento: 5, unos temporales: 17.0
Elemento: 6, unos temporales: 19.0
Elemento: 7, unos temporales: 20.0
Elemento: 8, unos temporales: 21.0
Elemento: 9, unos temporales: 23.0
indice_elemento 0
indice_elemento 1
indice_elemento 2
indice_elemento 3
indice_elemento 4
indice_elemento 5
indice_elemento 6
indice_elemento 7
indice_elemento 8
indice_elemento 9
indice_elemento 0
indice_elemento 1
indice_elemento 2
indice_elemento 3
indice_elemento 4
indice_elemento 

indice_elemento: 9 diferentes temp indices [10, 13, 24, 28, 31] dimensinoes utilizadas: 1
Se muto la dimension: 13
El elemento de la poblacion %s mutado es: 
[0 0 1 0 1 1 0 1 1 0 1 1 1 0 1 0 1 1 0 0 0 0 1 1 1 1 0 1 1 0 0 1 1 1 0 1 0
 0 1 1]
Entre al primer while
Diferentes temp cantidad 5
indice_elemento: 9 diferentes temp indices [10, 13, 24, 28, 31] dimensinoes utilizadas: 2
Se muto la dimension: 24
El elemento de la poblacion %s mutado es: 
[0 0 1 0 1 1 0 1 1 0 1 1 1 0 1 0 1 1 0 0 0 0 1 1 0 1 0 1 1 0 0 1 1 1 0 1 0
 0 1 1]
Entre al primer while
Diferentes temp cantidad 5
indice_elemento: 9 diferentes temp indices [10, 13, 24, 28, 31] dimensinoes utilizadas: 3
Se muto la dimension: 28
El elemento de la poblacion %s mutado es: 
[0 0 1 0 1 1 0 1 1 0 1 1 1 0 1 0 1 1 0 0 0 0 1 1 0 1 0 1 0 0 0 1 1 1 0 1 0
 0 1 1]
Entre al primer while
Diferentes temp cantidad 5
indice_elemento: 9 diferentes temp indices [10, 13, 24, 28, 31] dimensinoes utilizadas: 4
Se muto la dimension: 31
El elemento de 

Elemento: 0, unos temporales: 14.0
Elemento: 1, unos temporales: 23.0
Elemento: 2, unos temporales: 16.0
Elemento: 3, unos temporales: 18.0
Elemento: 4, unos temporales: 18.0
Elemento: 5, unos temporales: 18.0
Elemento: 6, unos temporales: 16.0
Elemento: 7, unos temporales: 21.0
Elemento: 8, unos temporales: 22.0
Elemento: 9, unos temporales: 20.0
Elemento: 0, unos temporales: 14.0
Elemento: 1, unos temporales: 23.0
Elemento: 2, unos temporales: 16.0
Elemento: 3, unos temporales: 18.0
Elemento: 4, unos temporales: 18.0
Elemento: 5, unos temporales: 18.0
Elemento: 6, unos temporales: 16.0
Elemento: 7, unos temporales: 21.0
Elemento: 8, unos temporales: 22.0
Elemento: 9, unos temporales: 20.0
indice_elemento 0
indice_elemento 1
indice_elemento 2
indice_elemento 3
indice_elemento 4
indice_elemento 5
indice_elemento 6
indice_elemento 7
indice_elemento 8
indice_elemento 9
indice_elemento 0
indice_elemento 1
indice_elemento 2
indice_elemento 3
indice_elemento 4
indice_elemento 5
indice_eleme

Diferentes temp cantidad 5
indice_elemento: 0 diferentes temp indices [2, 16, 20, 31, 34] dimensinoes utilizadas: 3
Se muto la dimension: 31
El elemento de la poblacion %s mutado es: 
[0 0 1 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 1 0 1 1 1 0 0 0 0 0
 0 0 1]
Entre al primer while
Diferentes temp cantidad 5
indice_elemento: 0 diferentes temp indices [2, 16, 20, 31, 34] dimensinoes utilizadas: 4
Se muto la dimension: 34
El elemento de la poblacion %s mutado es: 
[0 0 1 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 1 0 1 1 1 0 0 1 0 0
 0 0 1]


Me encuentro trabajando en el elmento 1 de 10 totales
Elemento:
[0 1 0 0 0 0 1 0 0 1 0 0 0 1 0 1 1 1 0 0 0 1 1 0 0 1 0 1 0 0 1 0 0 0 1 0 0
 0 0 0]
Tiene una probabilidad de error de: 0.24504280708
Para el elemento 1 se mutan un total máximo de M=10.0 dimensiones
Cantidad de dimensiones a mutar específicas para ese elemento: 5
El elemento mejor es: 
[0 0 1 0 1 1 1 0 1 1 1 0 1 0 1 1 0 0 0 0 1 1 0 0 0 1 0 1 1 1 1 1 0 1 1 0 0
 0 0 1]
El eleme

Elemento: 0, unos temporales: 12.0
Elemento: 1, unos temporales: 20.0
Elemento: 2, unos temporales: 15.0
Elemento: 3, unos temporales: 17.0
Elemento: 4, unos temporales: 17.0
Elemento: 5, unos temporales: 21.0
Elemento: 6, unos temporales: 15.0
Elemento: 7, unos temporales: 20.0
Elemento: 8, unos temporales: 25.0
Elemento: 9, unos temporales: 17.0
Elemento: 0, unos temporales: 12.0
Elemento: 1, unos temporales: 20.0
Elemento: 2, unos temporales: 15.0
Elemento: 3, unos temporales: 17.0
Elemento: 4, unos temporales: 17.0
Elemento: 5, unos temporales: 21.0
Elemento: 6, unos temporales: 15.0
Elemento: 7, unos temporales: 20.0
Elemento: 8, unos temporales: 25.0
Elemento: 9, unos temporales: 17.0
indice_elemento 0
indice_elemento 1
indice_elemento 2
indice_elemento 3
indice_elemento 4
indice_elemento 5
indice_elemento 6
indice_elemento 7
indice_elemento 8
indice_elemento 9
indice_elemento 0
indice_elemento 1
indice_elemento 2
indice_elemento 3
indice_elemento 4
indice_elemento 5
indice_eleme

...

Se muto la dimension: 2
El elemento de la poblacion %s mutado es: 
[1 1 0 0 0 1 0 1 0 1 0 0 1 0 0 1 0 0 1 1 0 1 0 1 0 1 0 0 1 1 1 0 0 1 1 0 1
 0 1 0]
Entre al primer while
Diferentes temp cantidad 5
indice_elemento: 5 diferentes temp indices [2, 8, 13, 19, 27] dimensinoes utilizadas: 1
Se muto la dimension: 8
El elemento de la poblacion %s mutado es: 
[1 1 0 0 0 1 0 1 1 1 0 0 1 0 0 1 0 0 1 1 0 1 0 1 0 1 0 0 1 1 1 0 0 1 1 0 1
 0 1 0]
Entre al primer while
Diferentes temp cantidad 5
indice_elemento: 5 diferentes temp indices [2, 8, 13, 19, 27] dimensinoes utilizadas: 2
Se muto la dimension: 13
El elemento de la poblacion %s mutado es: 
[1 1 0 0 0 1 0 1 1 1 0 0 1 1 0 1 0 0 1 1 0 1 0 1 0 1 0 0 1 1 1 0 0 1 1 0 1
 0 1 0]
Entre al primer while
Diferentes temp cantidad 5
indice_elemento: 5 diferentes temp indices [2, 8, 13, 19, 27] dimensinoes utilizadas: 3
Se muto la dimension: 19
El elemento de la poblacion %s mutado es: 
[1 1 0 0 0 1 0 1 1 1 0 0 1 1 0 1 0 0 1 0 0 1 0 1 0 1 0 0 1 1 1 0 0 1 1

Elemento: 1, unos temporales: 14.0
Elemento: 2, unos temporales: 15.0
Elemento: 3, unos temporales: 18.0
Elemento: 4, unos temporales: 18.0
Elemento: 5, unos temporales: 18.0
Elemento: 6, unos temporales: 14.0
Elemento: 7, unos temporales: 19.0
Elemento: 8, unos temporales: 22.0
Elemento: 9, unos temporales: 15.0
Elemento: 0, unos temporales: 12.0
Elemento: 1, unos temporales: 14.0
Elemento: 2, unos temporales: 15.0
Elemento: 3, unos temporales: 18.0
Elemento: 4, unos temporales: 18.0
Elemento: 5, unos temporales: 18.0
Elemento: 6, unos temporales: 14.0
Elemento: 7, unos temporales: 19.0
Elemento: 8, unos temporales: 22.0
Elemento: 9, unos temporales: 15.0
indice_elemento 0
indice_elemento 1
indice_elemento 2
indice_elemento 3
indice_elemento 4
indice_elemento 5
indice_elemento 6
indice_elemento 7
indice_elemento 8
indice_elemento 9
indice_elemento 0
indice_elemento 1
indice_elemento 2
indice_elemento 3
indice_elemento 4
indice_elemento 5
indice_elemento 6
indice_elemento 7
indice_elem

Elemento: 0, unos temporales: 11.0
Elemento: 1, unos temporales: 15.0
Elemento: 2, unos temporales: 13.0
Elemento: 3, unos temporales: 15.0
Elemento: 4, unos temporales: 15.0
Elemento: 5, unos temporales: 13.0
Elemento: 6, unos temporales: 11.0
Elemento: 7, unos temporales: 16.0
Elemento: 8, unos temporales: 23.0
Elemento: 9, unos temporales: 14.0
Elemento: 0, unos temporales: 11.0
Elemento: 1, unos temporales: 15.0
Elemento: 2, unos temporales: 13.0
Elemento: 3, unos temporales: 15.0
Elemento: 4, unos temporales: 15.0
Elemento: 5, unos temporales: 13.0
Elemento: 6, unos temporales: 11.0
Elemento: 7, unos temporales: 16.0
Elemento: 8, unos temporales: 23.0
Elemento: 9, unos temporales: 14.0
indice_elemento 0
indice_elemento 1
indice_elemento 2
indice_elemento 3
indice_elemento 4
indice_elemento 5
indice_elemento 6
indice_elemento 7
indice_elemento 8
indice_elemento 9
indice_elemento 0
indice_elemento 1
indice_elemento 2
indice_elemento 3
indice_elemento 4
indice_elemento 5
indice_eleme

/Users/Azeem/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in double_scalars



Entre al primer while
Diferentes temp cantidad 4
indice_elemento: 1 diferentes temp indices [13, 16, 30, 39] dimensinoes utilizadas: 0
Se muto la dimension: 13
El elemento de la poblacion %s mutado es: 
[0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0
 0 0 1]
Entre al primer while
Diferentes temp cantidad 4
indice_elemento: 1 diferentes temp indices [13, 16, 30, 39] dimensinoes utilizadas: 1
Se muto la dimension: 16
El elemento de la poblacion %s mutado es: 
[0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0
 0 0 1]
Entre al primer while
Diferentes temp cantidad 4
indice_elemento: 1 diferentes temp indices [13, 16, 30, 39] dimensinoes utilizadas: 2
Se muto la dimension: 30
El elemento de la poblacion %s mutado es: 
[0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0
 0 0 1]
Entre al primer while
Diferentes temp cantidad 4
indice_elemento: 1 diferentes temp indices [13, 16, 30, 39] dimensinoes utilizadas: 3
Se mut

In [20]:
criterio_A

[]

In [22]:
resultados_rendimiento

[[0.29679412677993167, 0.23805272146564344, 0.1989405802244599]]

In [23]:
resultados_score

[[0.7106862440668944, 0.73149092844785524, 0.74519806591846693]]

In [24]:
resultados_dimensiones_activas

[[array([0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]),
  array([0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0,
         0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  array([0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
         0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])]]

In [25]:
resultados_indices_dimensiones_activas

[[(array([ 5,  6,  7, 10, 12, 15, 16, 25, 27, 29, 30, 39]),),
  (array([ 4,  7, 12, 14, 19, 21, 24, 25, 27]),),
  (array([ 4,  6,  7, 14, 19, 27, 34]),)]]

In [21]:
criterio_C

[]

In [31]:
criterio_A=np.where(M_E_A[:,4]>=.6)
criterio_A=np.concatenate(criterio_A).ravel().tolist()

In [35]:
int(len(criterio_A))<5

True

In [33]:
len(criterio_A)

0

In [34]:
len(criterio_C)<5

True

In [18]:
registro_rendimiento_aceptados

[0]

In [20]:
M_M_T[m_m]

array([0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1])